<h2> Importing </h2>

In [11]:
import pandas as pd
import numpy as np
import requests
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime

<h2> Initialization </h2>

In [38]:
url="https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks"
table_name='largest_banks'
path=r'C:\Users\GPU-TECH\Desktop\Importantجدا\Banks_Projects_ETL\largest_banks.csv'
db_name='Banks.db'
log_file='code_log.txt'

<h2>Logging</h2>

In [39]:
def log_progress(message):
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now=datetime.now()
    timestamp=now.strftime(timestamp_format)
    with open('log_file','a') as f:
        f.write(timestamp+','+message+'\n')

<h2>Extract </h2>

In [40]:
def extract(url):
    try:
        html_data=requests.get(url).text
        data=BeautifulSoup(html_data,'html.parser')
        tables=data.find_all('tbody')
        rows=tables[1].find_all('tr')
        data_list=[]
        for row in rows:
            col=row.find_all('td')
            if len(col)!=0:
                data_dict={
                    'Bank name':col[1].text.strip(),
                    'MC_USD_Billion':col[2].text.strip()
                }
                data_list.append(data_dict)
        df=pd.DataFrame(data_list)
        return df   
    except:
        log_progress("an error occured during data extraction")
        
extract(url)

,Bank name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


<h2>Transform </h2>

In [49]:
def transform(df):
    exchange_rate = {
    'GBP': 0.73,  # Sample exchange rate for GBP
    'EUR': 0.82,  # Sample exchange rate for EUR
    'INR': 74.95  # Sample exchange rate for INR
    }
    df['MC_USD_Billion']=pd.to_numeric(df['MC_USD_Billion'].str.replace(',',''))
    # Adding columns MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    df['MC_GBP_Billion'] = [np.round(float(x) * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(float(x) * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(float(x) * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]
    

    return df

<h2>Loading_to_csv</h2>

In [50]:
def load_to_csv(df,path):
    df.to_csv(path,index=False)

<h2>Loading_to_db</h2>

In [51]:
def load_to_db(df,sql_connection,table_name,):
    df.to_sql(table_name,sql_connection,if_exists='replace')

<h2>Run Query </h2>

In [54]:
def run_query(query_statement,sql_connection):
    query_output=pd.read_sql(query_statement,sql_connection)
    return query_output

<h2>Function Calls</h2>

In [56]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect(db_name)

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

# Running the Query
query_statement = f"SELECT * FROM {table_name}"  # Modify the table name to match the one used in the database
result = run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()